# AM3strokes

#### Project description:
_A 3 strokes vertical apparent motion is presented left or right of fixation. <br />
Participants are instructed to saccadeto saccade to the perceived last stroke._

#### Hypothesis: 
_Participants mislocalize the 3rd stroke in the direction of the motion_

#### Eye movement data analysis:

- [x] Step 1. Extract time series
- [X] Step 2. Extract saccades for each trials

#### Step 1: extract time series

In [1]:
# Imports
import os
import numpy as np
import glob
import pandas as pd
from mat4py import loadmat
from sac_utils import vecvel, microsacc_merge, saccpar, isincircle
import ipdb

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

In [2]:
# Define parameters
num_trials = 140                       # number of trials per run
sampling_rate = 1000                   # eyetrakcing sampling rate
velocity_th = 1.5                      # velocity sd threshold
min_dur = 20                           # threshold minimum duration
merge_interval = 20                    # interval between saccade events
cor_sac_onset_th = 100                 # corrective saccade onset threshold (inferior or equal in ms)
fix_trial_end_soa = 800                # duration between fixation offset and trial end for blink checking 
fix_area_rad = 2                       # saccade onset position tolerance area in dva
sac_area_rad = 3                       # saccade landing area tolerance in dva
sac_lat_min = 50                       # saccade latency minimum duration
sac_lat_max = 400                      # saccade latency maximum duration

In [39]:
# Define folders
base_dir = '/home/mszinte/codes/am3strokes/'
data_dir = '/home/mszinte/codes/am3strokes/data'
subject = 'sub-17'
subject_num = subject[4:]
fig_dir = '{data_dir}/{subject}/figures'.format(data_dir=data_dir, subject=subject)

In [40]:
# Define data filenames
data_events = sorted(glob.glob('{}/{}/eyetrack/*.tsv'.format(data_dir,subject)))
num_run = len(data_events)

data_eyetrack = sorted(glob.glob('{}/{}/add/*.edf'.format(data_dir,subject)))
data_mat = sorted(glob.glob('{}/{}/add/*.mat'.format(data_dir,subject)))

In [41]:
# Create message and data files
for run in data_eyetrack:
    
    if not os.path.exists(run.replace('.edf','.msg')):
        os.system('edf2asc {} -e -y'.format(run))
        os.rename(run.replace('.edf','.asc'),run.replace('.edf','.msg'))

    if not os.path.exists(run.replace('.edf','.dat')):
        os.system('edf2asc {} -s -miss -1.0 -y'.format(run))
        os.rename(run.replace('.edf','.asc'),run.replace('.edf','.dat'))


EDF2ASC: EyeLink EDF file -> ASCII (text) file translator
EDF2ASC version 4.2.1.0 Linux   standalone Jun 18 2021 
(c)1995-2021 by SR Research, last modified Jun 18 2021

processing file /home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-01_eyetrack.edf 
loadEvents = 1
Preamble of file /home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-01_eyetrack.edf
| DATE: Thu Apr 21 15:42:53 2022                                              |
| TYPE: EDF_FILE BINARY EVENT SAMPLE TAGGED                                   |
| VERSION: EYELINK II 1                                                       |
| SOURCE: EYELINK CL                                                          |
| EYELINK II CL v5.12 May 12 2017                                             |
| CAMERA: Eyelink GL Version 1.2 Sensor=AI7                                   |
| SERIAL NUMBER: CLG-BAF38                                                    |
| CAMERA_CONFIG: BAF38200.SCD           

In [42]:
# Collect MSG data
msg_outputs = ['trial_onset', 'trial_offset', 'check_fix_onset', 'fix_onset', 'fix_offset', 
               's1_onset', 's1_offset', 's2_onset', 's2_offset', 's3_onset', 
               's3_offset']
 
for msg_output in msg_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(msg_output))

t_run = 0
for run in data_eyetrack:
    
    msgfid = open(run.replace('.edf','.msg'))
    first_last_time, first_time, last_time = False, False, False

    while not first_last_time:
        line_read = msgfid.readline()

        if not line_read == '':
            la = line_read.split()
    
            if len(la) > 2:
                if la[2] == 'RECORD_START' and not first_time: 
                    first_time = True
                if la[2] == 'RECORD_STOP' and not last_time:
                    last_time = True
            if len(la) > 4:
                if la[2] == 'trial' and la[4]=='started':
                    trial_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'trial' and la[4]=='ended':
                    trial_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'fix' and la[4]=='onset':
                    fix_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 'fix' and la[4]=='offset':
                    fix_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's1' and la[4]=='onset':
                    s1_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's1' and la[4]=='offset':
                    s1_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's2' and la[4]=='onset':
                    s2_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's2' and la[4]=='offset':
                    s2_offset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's3' and la[4]=='onset':
                    s3_onset[int(la[3])-1 + (t_run)*num_trials] = float(la[1])
                if la[2] == 's3' and la[4]=='offset':
                    s3_offset[int(la[3])-1 + (t_run)*num_trials,] = float(la[1])
    
        if first_time and last_time:
            first_last_time = True
            msgfid.close();
    t_run += 1

# create events dataframe
for run_num, run in enumerate(data_events):
    df_run = pd.read_csv(run, sep="\t")
    if run_num  > 0 :
        df_events = pd.concat([df_events, df_run])
    else :
        df_events = df_run

msg_dict = {}
for msg_output in msg_outputs:
    eval("msg_dict.update({'%s':%s})"%(msg_output,msg_output))
    
msg_dict.update({'fix_duration': fix_offset-fix_onset})
msg_dict.update({'s1_duration': s1_offset-s1_onset})
msg_dict.update({'s2_duration': s2_offset-s2_onset})
msg_dict.update({'s3_duration': s3_offset-s3_onset})

df_msg = pd.DataFrame(msg_dict)
df_all = pd.concat([df_events.reset_index(drop=True),
                    df_msg.reset_index(drop=True)], axis=1)

In [43]:
df_all

onset  duration  run_number  trial_number  trial_sequence  \
0    19378.22593  1.875654           1             1               6   
1    19380.82140  2.058853           1             2               7   
2    19383.41016  1.859168           1             3               4   
3    19385.63910  1.958832           1             4               3   
4    19387.95910  1.837921           1             5               6   
..           ...       ...         ...           ...             ...   
835  21747.16639  1.799008           6           136               6   
836  21749.31524  1.968981           6           137               6   
837  21751.79399  1.819058           6           138               5   
838  21753.96288  1.939006           6           139               2   
839  21756.26173  1.879010           6           140               4   

     saccade_direction  fixation_duration  post_stim_duration  \
0                    1                  8                   2   
1                    2                 10                   5   
2                    2                  3                   5   
3                    2                  8                   5   
4                    1                  9                   1   
..                 ...                ...                 ...   
835                  1                  7                   1   
836                  1                  3                   6   
837                  2                  1                   5   
838                  2                  7                   5   
839                  2                  4                   5   

     spatial_jitter_x  spatial_jitter_y  ...   s1_onset  s1_offset   s2_onset  \
0                  11                 2  ...  4722580.0  4722581.0  4722611.0   
1                   4                 6  ...  4725209.0  4725209.0  4725238.0   
2                   6                 1  ...  4727658.0  4727659.0        0.0   
3                   8                 2  ...  4729987.0  4729987.0        0.0   
4                   8                 5  ...  4732326.0  4732326.0  4732356.0   
..                ...               ...  ...        ...        ...        ...   
835                 6                 4  ...  7091478.0  7091478.0  7091508.0   
836                 8                 7  ...  7093548.0  7093548.0  7093578.0   
837                 1                 3  ...  7095985.0  7095985.0        0.0   
838                 2                 6  ...  7098274.0  7098275.0        0.0   
839                 5                 6  ...  7100513.0  7100513.0        0.0   

     s2_offset   s3_onset  s3_offset  fix_duration  s1_duration  s2_duration  \
0    4722611.0  4722641.0  4722641.0         849.0          1.0          0.0   
1    4725239.0  4725269.0  4725269.0        1039.0          0.0          1.0   
2          0.0        0.0        0.0         840.0          1.0          0.0   
3          0.0        0.0        0.0         941.0          0.0          0.0   
4    4732356.0  4732386.0  4732386.0         820.0          0.0          0.0   
..         ...        ...        ...           ...          ...          ...   
835  7091508.0  7091538.0  7091538.0         780.0          0.0          0.0   
836  7093578.0  7093608.0  7093608.0         951.0          0.0          0.0   
837        0.0        0.0        0.0         802.0          0.0          0.0   
838        0.0        0.0        0.0         921.0          1.0          0.0   
839        0.0        0.0        0.0         861.0          0.0          0.0   

     s3_duration  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
..           ...  
835          0.0  
836          0.0  
837          0.0  
838          0.0  
839          0.0  

[840 rows x 25 columns]

#### Step 2: extract saccades for each trials

In [44]:
sac_outputs = [  'miss_time_trial', 'blink_trial', 'no_saccade_trial', 'main_sac_trial', 'innacurate_sac_trial', 'early_sac_trial', 'late_sac_trial',
                 'sac_x_onset_trial', 'sac_x_offset_trial', 'sac_y_onset_trial', 'sac_y_offset_trial', 'sac_t_onset_trial', 'sac_t_offset_trial', 
                 'sac_dur_trial', 'sac_vpeak_trial', 'sac_dist_trial', 'sac_amp_trial', 'sac_dist_ang_trial','sac_lat_trial', 
                 'sac_amp_ang_trial', 'jitter_x_trial', 'jitter_y_trial', 'cor_sac_trial', 'cor_sac_x_onset_trial', 
                 'cor_sac_x_offset_trial', 'cor_sac_y_onset_trial', 'cor_sac_y_offset_trial', 'cor_sac_t_onset_trial', 'cor_sac_t_offset_trial', 
                 'cor_sac_dur_trial', 'cor_sac_vpeak_trial','cor_sac_dist_trial', 'cor_sac_amp_trial', 'cor_sac_dist_ang_trial', 
                 'cor_sac_amp_ang_trial', ]
 
for sac_output in sac_outputs:
    exec("{} = np.zeros(num_trials*num_run)".format(sac_output))

print('Saccade extraction:')
t_run = 0
first_fig = 0
for run,mat in zip(data_eyetrack,data_mat):
    
    print(run)
    # load const
    mat_dat = loadmat(mat)
    motion_jitter = np.array(mat_dat['config']['const']['motion_jitter'])
    fix_coord_ini = np.array(mat_dat['config']['const']['fix_coord'])
    mot_coord_x_range = np.array(mat_dat['config']['const']['mot_coord_x'])
    mot_coord_y_range = np.array(mat_dat['config']['const']['mot_coord_y'])
    scr_x_mid = np.array(mat_dat['config']['scr']['x_mid'])
    scr_y_mid = np.array(mat_dat['config']['scr']['y_mid'])
    mot_coord = np.zeros((mot_coord_x_range.shape[0],mot_coord_y_range.shape[0],2))
    for sac_dir_num, mot_coord_x in enumerate(mot_coord_x_range):
        for trial_seq_num, mot_coord_y in enumerate(mot_coord_y_range):     
            mot_coord[sac_dir_num,trial_seq_num] = [scr_x_mid+mot_coord_x, scr_y_mid-mot_coord_y]
    
    ppd = mat_dat['config']['const']['ppd']
    
    fix_area_rad_pix = ppd*fix_area_rad
    sac_area_rad_pix = ppd*sac_area_rad
    
    # load eyetrack data
    eye_data_run = np.genfromtxt(run.replace('.edf','.dat'), usecols=(0, 1, 2))

    for trial in np.arange(0,num_trials):
        
        # define trial
        miss_time, blink, no_saccade, main_saccade, corrective_saccade, num_sac = 0, 0, 0, 0, 0, 0
        trial_idx = trial + (t_run)*num_trials
        df_t = df_all.loc[(df_all.run_number == t_run+1) & (df_all.trial_number == trial+1)]
        trial_data_logic = np.logical_and(eye_data_run[:,0] >= float(df_t.trial_onset),
                                          eye_data_run[:,0] <= float(df_t.trial_offset))
        blink_data_logic = np.logical_and(eye_data_run[:,0] >= float(df_t.trial_onset),
                                          eye_data_run[:,0] <= float(df_t.fix_offset)+fix_trial_end_soa)

        # Missing data point detection
        if np.sum(np.diff(eye_data_run[trial_data_logic,0])>1000/sampling_rate) > 0:
            miss_time = 1
            miss_time_trial[trial_idx] = 1
                     
        # Blink detection
        if not miss_time:
            if np.sum(eye_data_run[blink_data_logic,1]== -1):
                blink = 1
                blink_trial[trial_idx] = 1

        # Main and corrective saccade detection
        if not miss_time and not blink:
            t, x, y = eye_data_run[trial_data_logic,0],eye_data_run[trial_data_logic,1],eye_data_run[trial_data_logic,2]
            vx, vy = vecvel(x,y,sampling_rate)
            sac = microsacc_merge(x,y,vx,vy,velocity_th,min_dur,merge_interval)
            ms = saccpar(sac)
                
            if np.isnan(ms[0,0]):
                #4 no saccade
                no_saccade = 1
                no_saccade_trial[trial_idx] = 1

                
            if not no_saccade:
                innacurate_sac = 1
                
                # Define fixation and saccade target position
                x_jitter = int(df_t.spatial_jitter_x)-1
                y_jitter = int(df_t.spatial_jitter_y)-1
                trial_sequence = int(df_t.trial_sequence)-1
                saccade_direction = int(df_t.saccade_direction)-1
                jitter_coord = np.array([motion_jitter[x_jitter],-motion_jitter[y_jitter]])
                fix_coord = fix_coord_ini + jitter_coord
                fix_pos_x, fix_pos_y = fix_coord[0], fix_coord[1]
                if trial_sequence <= 4:
                    last_stroke_coord = np.array(mot_coord[saccade_direction,trial_sequence]) + jitter_coord;
                else:
                    last_stroke_coord = np.array(mot_coord[saccade_direction,2]) + jitter_coord;
                sac_pos_x, sac_pos_y = last_stroke_coord[0], last_stroke_coord[1]
                
                while num_sac < ms.shape[0]:
                    
                    # Main saccade detection
                    fix_cor = isincircle(x[int(ms[num_sac,0])],y[int(ms[num_sac,0])],fix_pos_x,fix_pos_y,fix_area_rad_pix)
                    sac_cor = isincircle(x[int(ms[num_sac,1])],y[int(ms[num_sac,1])],sac_pos_x,sac_pos_y,sac_area_rad_pix)

                    if np.logical_and(fix_cor,sac_cor):
                        main_saccade = 1
                        innacurate_sac = 0
                        
                        sac_x_onset_trial[trial_idx] = (x[int(ms[num_sac,0])]-scr_x_mid)/ppd
                        sac_x_offset_trial[trial_idx] = (x[int(ms[num_sac,1])]-scr_x_mid)/ppd
                        sac_y_onset_trial[trial_idx] = -1*(y[int(ms[num_sac,0])]-scr_y_mid)/ppd
                        sac_y_offset_trial[trial_idx] = -1*(y[int(ms[num_sac,1])]-scr_y_mid)/ppd
                        sac_t_onset_trial[trial_idx] = t[int(ms[num_sac,0])]
                        sac_t_offset_trial[trial_idx] = t[int(ms[num_sac,1])]
                        sac_dur_trial[trial_idx] = ms[num_sac,2]
                        sac_lat_trial[trial_idx] = t[int(ms[num_sac,0])] - float(df_t.fix_offset)
                        sac_vpeak_trial[trial_idx] = ms[num_sac,3]/ppd
                        sac_dist_trial[trial_idx] = ms[num_sac,4]/ppd
                        sac_amp_trial[trial_idx] = ms[num_sac,6]/ppd
                        sac_dist_ang_trial[trial_idx] = ms[num_sac,5]
                        sac_amp_ang_trial[trial_idx] = ms[num_sac,7]
                        jitter_x_trial[trial_idx] = (jitter_coord[0])/ppd
                        jitter_y_trial[trial_idx] = -1*(jitter_coord[1])/ppd
                        
                        if sac_lat_trial[trial_idx] < sac_lat_min:
                            early_sac_trial[trial_idx] = 1
                            main_saccade = 0
                            
                        if sac_lat_trial[trial_idx] > sac_lat_max:
                            late_sac_trial[trial_idx] = 1
                            main_saccade = 0
                            
                    # Corrective saccade detection
                    if main_saccade and corrective_saccade == 0:
                        if t[int(ms[num_sac,0])] <= t[int(ms[num_sac-1,0])] + cor_sac_onset_th:
                            corrective_saccade = 1
                            
                            cor_sac_x_onset_trial[trial_idx] = (x[int(ms[num_sac,0])]-scr_x_mid)/ppd
                            cor_sac_x_offset_trial[trial_idx] = (x[int(ms[num_sac,1])]-scr_x_mid)/ppd
                            cor_sac_y_onset_trial[trial_idx] = -1*(y[int(ms[num_sac,0])]-scr_y_mid)/ppd
                            cor_sac_y_offset_trial[trial_idx] = -1*(y[int(ms[num_sac,1])]-scr_y_mid)/ppd
                            cor_sac_t_onset_trial[trial_idx] = t[int(ms[num_sac,0])]
                            cor_sac_t_offset_trial[trial_idx] = t[int(ms[num_sac,1])]
                            cor_sac_dur_trial[trial_idx] = ms[num_sac,2]
                            cor_sac_vpeak_trial[trial_idx] = ms[num_sac,3]/ppd
                            cor_sac_dist_trial[trial_idx] = ms[num_sac,4]/ppd
                            cor_sac_amp_trial[trial_idx] = ms[num_sac,6]/ppd
                            cor_sac_dist_ang_trial[trial_idx] = ms[num_sac,5]
                            cor_sac_amp_ang_trial[trial_idx] = ms[num_sac,7]
                    
                    num_sac += 1 
                    
                    main_sac_trial[trial_idx] = main_saccade
                    cor_sac_trial[trial_idx] = corrective_saccade
                    innacurate_sac_trial[trial_idx] = innacurate_sac    

    t_run +=1
print('Done')

saccade_dict = {}
for sac_output in sac_outputs:
    eval("saccade_dict.update({'%s':%s})"%(sac_output,sac_output))

df_saccade = pd.DataFrame(saccade_dict)
df_all = pd.concat([df_all.reset_index(drop=True),
                    df_saccade.reset_index(drop=True)], axis=1)

for run in data_eyetrack:
    os.remove(run.replace('.edf','.msg'))
    os.remove(run.replace('.edf','.dat'))

Saccade extraction:
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-01_eyetrack.edf
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-02_eyetrack.edf
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-03_eyetrack.edf
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-04_eyetrack.edf
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-05_eyetrack.edf
/home/mszinte/codes/am3strokes/data/sub-17/add/sub-17_task-AM3strokes_run-06_eyetrack.edf
Done


In [45]:
# Save dataframe
df_all.to_csv('{data_dir}/{subject}/add/{subject}_task-AM3strokes_data.csv'.format(data_dir=data_dir,subject=subject))